# Code to Scrap Data from Indeed.com

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import cloudscraper
import pandas as pd

In [2]:
def get_url(position, location):
    template = 'https://in.indeed.com/jobs?q={}&l={}'
    url = template.format(position, location)
    return url

In [3]:
print(get_url('python+developer', 'Mumbai'))
scraper = cloudscraper.create_scraper() 
response = scraper.get(get_url('python+developer', 'Mumbai'))
soup = BeautifulSoup(response.text, 'html.parser')
k = 0
listUrls = []
prefix = "https://in.indeed.com"
for i in soup.find_all('div', class_='css-tvvxwd ecydgvn1'):
    if(k == 0):
        k = k + 1;
        continue
    listUrls.append(prefix+i.find('a').get('href'))
print(listUrls)

https://in.indeed.com/jobs?q=python+developer&l=Mumbai
['https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=10', 'https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=20', 'https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=30', 'https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=40', 'https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=10']


In [4]:
def main(pos, loc):
    position = []
    company_name = []
    location = []
    salary = []
    description = []
    applyLinks = []
    
    for url in listUrls:
        response = scraper.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        print(url)
        
        temp = soup.find_all('div', 'css-1m4cuuf e37uo190')
        for i in temp:
            position.append(i.text.strip())
        
        temp = soup.find_all('span','companyName')
        for i in temp:
            company_name.append(i.text.strip())
            
        temp = soup.find_all('div','companyLocation')
        for i in temp:
            location.append(i.text.strip())
        
        temp = soup.find_all('div','heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly')
        for i in temp:
            salary.append(i.text)
        
        temp = soup.find_all('div','job-snippet')
        for i in temp:
            description.append(i.text)
        
        prefix = "https://in.indeed.com"
        applyUrls = []
        temp = soup.find_all('h2', class_='jobTitle')
        for i in temp:
            applyUrls.append(prefix + i.find('a').get('href'))
          
        for i in applyUrls:
            response = scraper.get(i)
            soup = BeautifulSoup(response.text, 'html.parser')
            if(soup.find('div', class_='icl-u-xs-hide icl-u-lg-block icl-u-lg-textCenter') != None):
                applyLinks.append(soup.find('div', class_='icl-u-xs-hide icl-u-lg-block icl-u-lg-textCenter').find('a').get('href'))
            else:
                applyLinks.append(soup.find('div', class_='ia-IndeedApplyButton').find('span').get('data-indeed-apply-posturl'))

    d = {'Position':position, 'Company':company_name, 'Location':location, 'Description':description, 'Apply Link':applyLinks}
    print(len(position))
    print(len(company_name))
    print(len(location))
    print(len(description))
    print(len(applyLinks))
    
    return pd.DataFrame(d)

In [5]:
df = main('python+developer', 'Mumbai')

https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=10
https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=20
https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=30
https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=40
https://in.indeed.com/jobs?q=python+developer&l=Mumbai&start=10
75
75
75
75
75


In [6]:
df

,Position,Company,Location,Description,Apply Link
0,Python/Django Developer,Kokilaben Hospital,"Mumbai, Maharashtra",\nFull-Stack Product & Features Development us...,http://muffit/process-indeedapply
1,Python Developer,Focskill pvt.ltd,"Mumbai, Maharashtra",\nCloud Knowledge(AWS Lambda/s3/SQS/RDS) (Requ...,http://muffit/process-indeedapply
2,Python Developer with AWS,InKnowTech Private Limited,"Mumbai, Maharashtra","\nJob Types: Full-time, Regular / Permanent.\n...",http://muffit/process-indeedapply
3,Senior Python Developernew,Icapotech Pvt Ltd,"Mumbai, Maharashtra",\nYou should have hands-on knowledge in system...,http://muffit/process-indeedapply
4,Python Developer,Neural IT Pvt. Ltd.,"Airoli, Navi Mumbai, Maharashtra",\nExpertise in at least one popular Python fra...,http://muffit/process-indeedapply
...,...,...,...,...,...
70,Python Developer,SySpree Solutions,"Mumbai, Maharashtra",Job Features Job Category Web Developer Positi...,https://in.indeed.com/applystart?jk=d9488d734e...
71,Python Developer,Orcapod,"Mumbai, Maharashtra","\nProject Role Description :Design, build and ...",https://in.indeed.com/applystart?jk=e8de9833f3...
72,Sr. Python Developer,Atrina technologies,"Mumbai, Maharashtra",\nYou will need a practical approach to solvin...,http://muffit/process-indeedapply
73,Backend Developer (Springboot),Defacto Technologies Pvt Ltd,"Temporarily Remote in Mumbai, Maharashtra","\nConceptualize, plan and implement some excit...",http://muffit/process-indeedapply


In [7]:
df.to_csv('D:\data.csv')